BULL VERTICAL SCREENER
=======

We want to find high ROC credit bull PUT verticals within the S&P500 constituents list, in underlyings that comply with :

* Market cap > 'min_market_cap'.
* Average option volume > 'min_option_volume'.
* Iv rank > min_iv_rank     <===== TODO!!!!!
* minimum days to earnings > min_days_to_earnings (Wall Street Horizons subscription is needed)

After previous filter, scan option chains for remaining underlyings:

* Explore up to 'num_month_expiries' monthly expiries forward from current date.
* Scan combinations of strike prices starting from smallest strike to  (ATM - 'pct_under_px_range'% ) range. Zero explores all available strike combinations.
* Limit risk to 'max_risk' USD (which influences allowable distance between strikes).

Notes:
1. If notional value of a naked put strategy is less than 'max_risk' then switch to naked put strategy (do not buy long leg).
2. Order resulting strategies descending on 'Return on Capital' (MaxProfit / Margin ).
3. If 'ask_tws_load==True', user will be prompted to load strategies on TWS.
4. Only OTM options will be scaned (strikes below current price)


Get list of SP500 constituents, look for 'getSp500Constituents.py' in parent folders in order to refresh csv list file. Order descending by market cap, then sample first 'constituents_slice'

In [54]:
from ib_insync import *
import pandas as pd

import xml.etree.ElementTree as ET

from datetime import datetime, timezone, timedelta



#underlying filters
min_market_cap= 2000 #minimum market capital in USD Millions Dollars
constituents_slice = 2 # After minimum market cap ordering / filtering scan up to to this number of securities
min_option_volume = 10000 #minimum average daily option volume
min_iv_rank = 15 # min 52 weeks Implied Volatility Rank (%)
min_days_to_earnings = 45 #minimum days to next earnings report

#option strategy filters
pct_under_px_range = .20 # scan strikes under this % below market price for underlying 
num_month_expiries = 3 #monthly expires forward
max_loss = 2000 # max loss
min_profit = 200 #min profit
ask_tws_load =True  #ask wheter to load strategies


## SP500 Contituents scan 

Create a subset of S7P500 index constituents by applying pre defined 'min_market_cap' and 'constituents_slice' filters

In [55]:
df = pd.read_csv( '../sp500Constituents.csv', index_col = 'Symbol' )

print( df.columns )

sp500_constituents = df[ ['Market Cap'] ]

sp500_marketCap = sp500_constituents.sort_values( by = 'Market Cap', ascending=False, axis = 0 )

sp500_marketCap = sp500_marketCap[ : constituents_slice ]


Index(['Unnamed: 0', 'Name', 'Sector', 'Price', 'Price/Earnings',
       'Dividend Yield', 'Earnings/Share', '52 Week Low', '52 Week High',
       'Market Cap', 'EBITDA', 'Price/Sales', 'Price/Book', 'SEC Filings'],
      dtype='object')


**Connecto to IB**

In [60]:
#start ib_insync loop
util.startLoop()

ib = IB()

if ib.isConnected() == False:
    ib.connect('127.0.0.1', 7497, clientId=21, readonly=True )


**Request contracts for SP500 subset**

**Create contract for each stock in the subset**

In [62]:

Contracts = [ Stock( s, 'SMART', currency='USD' ) for s in sp500_marketCap.index ]


**Request contract validation**

In [63]:
ib.qualifyContracts( *Contracts )

[Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'),
 Stock(conId=208813719, symbol='GOOGL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='GOOGL', tradingClass='NMS')]

**Get the tickers for each of the stock contracts. Requesting each ticker can take up to 11 seconds.**

In [64]:
    #105: avOptionVolume
tickers = [ ib.reqMktData( c, '105') for c in Contracts ]
print( ' {} tickers returned out of {} requested'.format( len(tickers), len(Contracts) ) )

#wait for tickers to be filled
ib.sleep(2)

#[tickers] = [  ib.reqTickers( *Contracts )]

 2 tickers returned out of 2 requested


True

**Apply option volume / nearing filters using ticker option volume data**


In [68]:
tickers

[Ticker(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), time=datetime.datetime(2020, 6, 11, 3, 22, 11, 479938, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=351.0, lastSize=9, volume=416665, open=347.97, high=354.77, low=346.09, close=343.99, halted=0.0, avOptionVolume=585563),
 Ticker(contract=Stock(conId=208813719, symbol='GOOGL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='GOOGL', tradingClass='NMS'), time=datetime.datetime(2020, 6, 11, 3, 22, 11, 481693, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=1460.35, lastSize=1, volume=15881, open=1461.51, high=1472.77, low=1454.35, close=1452.08, halted=0.0, ticks=[TickData(time=datetime.datetime(2020, 6, 11, 3, 22, 11, 481693, tzinfo=datetime.timezone.utc), tickType=1, price=-1.0, size=0), TickData(time=datetime.datetime(2020, 6, 11, 3, 22, 11, 481693,

In [69]:
# apply option volume filter,  create 'symbol' : 'ticker dictionary'
filteredTickers = { ticker.contract.symbol: ticker for ticker in tickers if ticker.avOptionVolume > min_option_volume }

#create 'symbol' : 'contract' dictionary
filteredContracts = { symbol: ticker.contract for ( symbol, ticker ) in filteredTickers.items() }

print( '{} contracts remaining after average option volume filter'.format( len( filteredContracts ) ) )

1 contracts remaining after average option volume filter


**Remove contracts with nearing earnings report**

In [70]:
# minimum earning clear date
clearDateToEarnings = datetime.now() + timedelta( days = min_days_to_earnings )

earningsFiltered = []

print( 'minimum earning clear date requirement {}'.format( clearDateToEarnings) )

for ( symbol, contract ) in filteredContracts.items():
    #get calendar from wall street horizons
    root = ET.fromstring( ib.reqFundamentalData( contract, 'CalendarReport' ) )

    for earningsNode in root.iter('Earnings'):
        dateNode = earningsNode.find( 'Date' )

        if dateNode != None:
            nextEarnigsDate = pd.to_datetime( dateNode.text )
            if nextEarnigsDate < clearDateToEarnings:
                #tag for removal
                earningsFiltered.append( symbol )
                print( '{} next earnings date: {} no pass'.format( symbol, nextEarnigsDate ) )
            else:
                print( '{} next earnings date: {} pass'.format(symbol, nextEarnigsDate ) )
        else:
            print( 'missing next earnings date for {}'.format(symbol) )

#remove tagged
for key in earningsFiltered:
    filteredTickers.pop( key, None )
    filteredContracts.pop( key, None )


minimum earning clear date requirement 2020-07-25 22:24:46.044380
AAPL next earnings date: 2020-07-28 00:00:00 pass


**Get option chains for each contract, filter by 'SMART' exchange**

In [71]:
#get all chains
chains = { symbol : ib.reqSecDefOptParams( symbol, '', contract.secType, contract.conId) 
         for ( symbol, contract ) in filteredContracts.items() }

#leave only chains in 'SMART' exchange
chains = { symbol : [ chain for chain in chainList if chain.exchange == 'SMART' ] for ( symbol, chainList ) in chains.items() }

print( ' {} chains found of {} requested'.format( len(chains.keys()), len(filteredContracts.keys()) ) )



 1 chains found of 1 requested


In [72]:
newexpirations = [ exp for exp in chains['AAPL'][0].expirations if pd.to_datetime(exp) < option_expiration_limit ]


From this point on we can build all PUT  option contracts that meet our conditions (Naked puts / Bull Put Spreads).

Detail of implemented algorithm:
  
1. Iterate over stock option 'chains', select first chain.
  1. filter the expirations list, limit to up to 3 month expirations.
  2. filter strike list, limit to strikes below 'pct_under_px_range'
  3. Explore remaining strike / expiriation combinations in filtered option chains:
     1. **Chains / Expirations loop**. Iterate over chains/ expirations, select first chain/expiration (might be only one per chain):
        1.  **Leg 1 loop**. Iterate over strikes, select leg 1 strike.
            1. Combine selected strike with current expiration from outer loop.
            2. Build Option contract.
            3. Request market data (price) for 1st leg ontract.
            4. Fix leg 1 contract for going to inner loop.
                 1. **Leg 2 loop**. Iterate over strikes, select leg 2 strike
                     1. Is strike < combo leg 1 strike? No : Discard / iterate again, Yes: Select it.
                     2. Combine selected strike with current expiration from outer loop
                     3. Build Option contract.
                     4. Request market data (price) for 2nd leg contract.
                     5. Calculate COMBO price (max profit) using 1st and 2nd leg market data.
                     6. Is maximum profit > 'min_profit' from screener parameters? No: Discard.
                     7. Calculate maximum loss using 1st and 2nd leg strike distance.
                     8. Is maximum loss > 'max_risk' from screener parameters? Yes: Discard.
                     9. Create 'COMBO' order,  request 'whatif' for price / margin information
                    10. Save 'COMBO' order / legs / ROC etc in local **results** data frame.
            5. Continue loop to step 1 in **Leg 1 loop**.
        2. Save local **results** array into **combosByChain** dictionary.
        3. Continue loop to step 1 in **Chains / Expirations loop**.
  4. Results are stored in 'combosByContract' dictionary.

In [73]:
# prepare 'num_month_expiries' datetime object
curdate = datetime.now()
delta_forward = timedelta( weeks = num_month_expiries*4 )
option_expiration_limit = curdate + delta_forward


## Filter option chains

Filter option chains for filtered contracts according to:
* **'pct_under_px_range'** (percentage below price)  
* **'option_expiration_limit'** (option expiration)

In [74]:
# filter chains strikes / expirations:

adjustedContractChains = {}

for ( symbol, contract ) in filteredContracts.items():

    adjustedChains = []
    
    curChains = chains[ symbol ]
    curPrice = filteredTickers[ symbol ].marketPrice() # might need to replace by 'saved' value
        
    #usually only one chain ('SMART' exchange)
    for chain in curChains:
        
        #filter expirations ('option_expiration_limit')
        newexpirations = [ exp for exp in chain.expirations if pd.to_datetime(exp) < option_expiration_limit ]
        
        #filter strikes ('pct_under_px_range')
        newstrikes = [ s for s in chain.strikes if s > ( curPrice * ( 1 - pct_under_px_range ) ) and s <= curPrice ]

        #OptionChain is 'NamedTuple' (not mutable)
        adjustedChain = OptionChain( chain.exchange, 
                                    chain.underlyingConId,
                                    chain.tradingClass,
                                    chain.multiplier,
                                    newexpirations,
                                    newstrikes)
        #adjustedChain.strikes = newstrikes
        adjustedChains.append( adjustedChain )
    
    adjustedContractChains.update( { symbol : adjustedChains } )
    
print("strike / expiration option chain complete for {} chains".format( len(adjustedContractChains) ) )

strike / expiration option chain complete for 1 chains


### Combo screening

Create L1/L2 combo orders, calculate prices, filter by min_profit, max_loss, save into **combosByContract** / **combosByChain** dictionaries


In [ ]:
# screened contracts will be placed here, one list per contract
# { symbol : [ { expiration : dataframe }, ... ] }
combosByContract = {}
combosByChain = {}


# Create / Filter combos
for ( symbol, contract ) in filteredContracts.items():

    for chain in adjustedContractChains[ symbol ]:
            
        # ----- expirations loop -----
        for exp in chain.expirations:
            
            # results per expiration
            results = []

            #create contracts for all strikes
            comboContracts = [Option(contract.symbol, exp, strike, 'P', contract.exchange )
                    for strike in chain.strikes]

            comboContracts = ib.qualifyContracts( *comboContracts )

            # should be less than 100 requests (request limit)
            comboTickers = ib.reqTickers( *comboContracts )

            print( '{} tickers received for expiration {} in contract chain'.format( len(comboTickers), exp, symbol ) )

            # ---- leg1 loop ----
            for leg1Ticker in comboTickers:
                leg1Price = leg1Ticker.marketPrice()
                leg1Strike = leg1Ticker.contract.strike
                 # early filter leg1 by min_profit
                if( ( leg1Price * int(chain.multiplier) ) < min_profit ):
                    comboContracts.remove(leg1Ticker.contract)
                else:
                    #fix leg 1
                    leg1 = ComboLeg(conId=leg1Ticker.contract.conId, 
                                    ratio= 1, 
                                    action='SELL', 
                                    exchange=leg1Ticker.contract.exchange )
                    # ---- leg 2 loop ----
                    for leg2Ticker in comboTickers:
                        leg2Strike = leg2Ticker.contract.strike
                        if( leg2Ticker.contract.strike < leg1Strike ):
                            leg2Price = leg2Ticker.marketPrice()
                            #leg2 hedge should cost less than leg1
                            comboPrice = ( leg1Price - leg2Price ) * int(chain.multiplier)

                            if ( comboPrice < max_loss ) and \
                            ( comboPrice > min_profit ):

                                leg2 = ComboLeg( conId=leg2Ticker.contract.conId,  
                                                ratio=1,
                                                action='BUY',
                                                exchange=leg2Ticker.contract.exchange )

                                #create 'combo contract'
                                combo = Contract( symbol=leg1Ticker.contract.symbol,#check!! 
                                                 secType='BAG', 
                                                 currency='USD', 
                                                 exchange=leg1Ticker.contract.exchange, 
                                                 comboLegs=[leg1, leg2])

                                order = MarketOrder( action="BUY", 
                                                   totalQuantity = 1 )

                                #this method is blocking
                                orderState = ib.whatIfOrder( combo, order )

                                roc = comboPrice / float(orderState.maintMarginChange)

                                #model greeks might not be pupulated (check)
                                if leg1Ticker.modelGreeks != None :
                                    leg1IV = leg1Ticker.modelGreeks.impliedVol
                                    leg2IV = leg2Ticker.modelGreeks.impliedVol
                                else:
                                    leg1IV = float('nan')
                                    leg2IV = float('nan')



                                results.append( [ leg1Strike,
                                                   leg1Ticker.contract.conId,
                                                   leg1IV,
                                                   leg2Strike,
                                                   leg2Ticker.contract.conId,
                                                   leg2IV,
                                                   comboPrice,
                                                   float(orderState.maintMarginChange),
                                                   roc ] )


            combos_df = pd.DataFrame( data = results, columns=[ 'leg1Strike', 
                                                                'leg1ContractId',
                                                                'leg1IV',
                                                                'leg2Strike',
                                                                'leg2ContractId',
                                                                'leg2IV',
                                                                'maxProfit',
                                                                'margin',
                                                                'ROC' ] )

            print( '{} combos found for {} contract chain with expiration {}'.format( \
                len( combos_df.index ), symbol, exp ) )

            combosByChain.update( { exp : combos_df } )
                        
    combosByContract.update( { symbol : combosByChain } )

27 tickers received for expiration 20200612 in contract chain


## RESULTS

At this point we should have:

- **combosByContract** : dictionary, all combos per expiration grouped by contracts
- **combosByContract** : dictionary, all combos grouped by expiration


TODO:
- **IvRanks**: dictionary, ivRanks for each of the possible underlying contracts

It should be straightforward to analyze and visually represent the best strategies.

In [29]:
ib.disconnect()